In [ ]:
from google.colab import userdata
api_key=userdata.get('APIKEY')


In [ ]:
!pip install -q U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=api_key)
response=model.invoke("What is meant MCP?")
print(response)

content='MCP can stand for a few different things, depending on the context. Here are some of the most common meanings:\n\n**1. Microsoft Certified Professional (MCP):**\n\n*   This is the most common meaning. It refers to an individual who has passed one or more Microsoft certification exams. It\'s a foundational certification that demonstrates basic proficiency in a Microsoft technology. While Microsoft has evolved its certification program, the term "MCP" is still often used to refer to anyone who has achieved at least one Microsoft certification.\n\n**2. Main Control Program (MCP):**\n\n*   This refers to the operating system used by Burroughs Corporation (later Unisys) mainframe computers. It was an early and influential operating system known for its security and use of high-level languages.\n\n**3. Massively Concurrent Processing (MCP):**\n\n*   In computer science, this refers to architectures or systems designed to handle a large number of simultaneous processes.\n\n**4. Maste

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.3 MB/s eta 0:00:00


In [ ]:
!pip install langgraph

In [ ]:
! pip install langgraph tavily-python

In [ ]:
from typing import List
from langchain_core.messages import HumanMessage, AIMessage , BaseMessage
from langchain_core.tools import tool
from tavily import TavilyClient
from langgraph.graph import MessageGraph,END
from google.colab import userdata
search_key=userdata.get('Search_key')
tavily = TavilyClient(api_key=search_key)

@tool(description="Extracted Structured data like profile , skills , experience from resume")
def Parse_Resume(text) :
  prompt=f"Extract structured info like skills, roles, years of experience from this resume:\n\n{text}"
  return model.invoke([HumanMessage(content=prompt)]).content

@tool(description="Searching")
def Search_tavily(query):
  results=tavily.search(query)
  print(results)
  return results["results"]

#Nodes
def Profile_node(state: List[BaseMessage])-> List[BaseMessage]:
  resume=state[-1].content
  profile= Parse_Resume.invoke(resume)
  return state+[AIMessage(content=f"Extracted Profile:{profile}")]

def job_search_node(state: List[BaseMessage])-> List[BaseMessage]:
  profile=state[-1].content
  query = model.invoke([HumanMessage(content=f"Based on this profile, write a job search query/description: \n{profile}, keep the query/description short for search engines")]).content
  jobs=Search_tavily.invoke(query)
  return state+[AIMessage(content=f"Found Jobd:\n {jobs}")]

def matcher_node(state: List[BaseMessage])-> List[BaseMessage]:
  profile=state[-2].content
  jobs=state[-1].content
  prompt=f"Given this profile:\n{profile} and recommend top 3 jobs from:\n{jobs}"
  ranked=model.invoke([HumanMessage(content=prompt)])
  return state + [AIMessage(content=f"Ranked Jobs:\n{ranked}")]

def cover_letter_node(state:List[BaseMessage])-> List[BaseMessage]:
  profile=state[-3].content
  jobs=state[-2].content
  ranked=state[-1].content
  prompt1=f"Write a cover letter for this job:{ranked} \n\n based on the profile{profile} "
  cover_letter=model.invoke([HumanMessage(content=prompt1)])
  return state+[AIMessage(content=f"Cover Letter:\n{cover_letter}")]

graph=MessageGraph()
graph.add_node("Profile",Profile_node)
graph.add_node("JobSearch",job_search_node)
graph.add_node("Matcher",matcher_node)
graph.add_node("CoverLetter",cover_letter_node)
graph.add_edge("Profile","JobSearch")
graph.add_edge("JobSearch","Matcher")
graph.add_edge("Matcher","CoverLetter")
graph.add_edge("CoverLetter",END)
graph.set_entry_point("Profile")
Career_agent=graph.compile()

Input_msg=HumanMessage(content="See, I am fresher with no experience and have done projects in cloud, Machinelearning and in GenAI")
response=Career_agent.invoke(Input_msg)

for msg in response:
  print({msg.content})















{'query': '**Option 1 (Shortest):**\n\n> Cloud Computing, Machine Learning, GenAI - Entry Level\n\n**Option 2 (Slightly More Specific):**\n\n> Entry-Level Cloud, ML, GenAI Roles - Fresher\n\n**Option 3 (Focus on Projects):**\n\n> Cloud, Machine Learning, GenAI Projects - Entry Level', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Top 10 Entry-Level Tech Jobs in 2025 - nextwork.org', 'url': 'https://www.nextwork.org/blog/top-10-entry-level-tech-jobs-in-2025', 'content': 'Discover the top 10 entry-level tech jobs in 2025, including data analysis, cloud computing, and cybersecurity. Learn skills, salaries, and hands-on projects to kickstart your tech career. ... AI/ML Junior Developers assist in creating machine learning models and AI algorithms. They help process data, fine-tune models, and integrate AI into', 'score': 0.78963995, 'raw_content': None}, {'title': 'All Generative AI entry-level jobs - aijobs.net', 'url': 'https://aijobs.net/list/generati